# Structured Outputs con Microsoft Agent Framework

## Descripción

En este ejercicio, aprenderás a crear agentes que producen **salidas estructuradas** en forma de objetos JSON que se ajustan a un esquema específico. Esto es fundamental cuando necesitas que la IA genere datos en un formato predecible y procesable.

### Casos de uso:
- Extracción de información clave de textos
- Generación de objetos Python desde texto
- Validación y transformación de datos
- Análisis de documentos con formato consistente
- Procesamiento de formularios y tickets

## Configuración inicial

### Cargar librerías

Importamos las bibliotecas necesarias. Usaremos **Pydantic** para definir los esquemas de datos.

In [5]:
import os
import asyncio
import json
from typing import List, Optional
from enum import Enum
from pydantic import BaseModel, Field
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

# GitHub Models client configuration
MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")


## Función auxiliar común

Esta función auxiliar será utilizada por los 2 ejercicios para ejecutar el agente con structured outputs.

In [6]:
async def run_structured_agent(
    instructions: str,
    user_input: str,
    response_format: type[BaseModel],
    agent_name: str = "structured_agent"
) -> BaseModel:
    """
    Función auxiliar para ejecutar un agente con salida estructurada.
    
    Args:
        instructions: Instrucciones para el agente
        user_input: Entrada del usuario
        response_format: Clase Pydantic que define el formato de salida
        agent_name: Nombre del agente
        
    Returns:
        Instancia del modelo Pydantic con los datos extraídos
    """
    client = OpenAIChatClient(
        model_id=MODEL_NAME,
        api_key=os.environ["GITHUB_TOKEN"],
        base_url="https://models.github.ai/inference"
    )
    
    async with ChatAgent(
        chat_client=client,
        name=agent_name,
        instructions=instructions,
        response_format=response_format
    ) as agent:
        response = await agent.run([user_input])
        # The AgentRunResponse object has the 'value' attribute containing the JSON dictionary
        return response_format.model_validate(response.value)


print("Función auxiliar definida correctamente")


Función auxiliar definida correctamente


In [7]:
import os

# Verificar si existe el token
token = os.environ.get("GITHUB_TOKEN")
if token:
    print(f"Token encontrado: {token[:10]}...")  # Muestra solo los primeros caracteres
else:
    print("❌ GITHUB_TOKEN no está configurado")

Token encontrado: ghu_yddGIV...


---

## Ejercicio 1: Extracción de información clave de texto

En este ejercicio, extraeremos información estructurada de un artículo de noticias. El agente identificará los elementos clave del texto y los organizará en un formato JSON predefinido.

In [8]:
# Define the schema for article information
class ArticleInfo(BaseModel):
    """Información estructurada extraída de un artículo"""
    title: str = Field(description="Título principal del artículo")
    summary: str = Field(description="Resumen breve del contenido")
    main_topic: str = Field(description="Tema principal")
    key_points: List[str] = Field(description="Puntos clave mencionados")
    entities: List[str] = Field(description="Personas, organizaciones o lugares mencionados")
    date_mentioned: Optional[str] = Field(None, description="Fecha mencionada en el artículo")
    sentiment: str = Field(description="Sentimiento general: positivo, neutral o negativo")


# Example text: news article
article_text = """
La Unión Europea anunció ayer un ambicioso plan para reducir las emisiones de carbono 
en un 55% para el año 2030. La presidenta de la Comisión Europea, Ursula von der Leyen, 
presentó el paquete de medidas en una conferencia de prensa en Bruselas.

El plan incluye inversiones masivas en energías renovables, con un enfoque especial en 
energía solar y eólica. Se destinarán 750 mil millones de euros para la transición verde. 
Además, se implementarán nuevas regulaciones para la industria automotriz, exigiendo que 
todos los vehículos nuevos sean eléctricos para 2035.

Los países miembros como Alemania, Francia y España han expresado su apoyo a la iniciativa. 
Sin embargo, algunos estados del este de Europa han manifestado preocupaciones sobre los 
costos de implementación. Expertos ambientalistas consideran que este es un paso crucial 
en la lucha contra el cambio climático.
"""

# Instructions for the agent
instructions_article = """Eres un experto en análisis de contenido y extracción de información.

Analiza el texto proporcionado y extrae:
- Un título descriptivo
- Un resumen conciso
- El tema principal
- Los puntos clave más importantes
- Nombres de personas, organizaciones y lugares
- Fechas mencionadas
- El sentimiento general del artículo

Sé preciso y objetivo. Responde SOLO con el JSON estructurado."""

# Run the agent
article_info = await run_structured_agent(
    instructions=instructions_article,
    user_input=article_text,
    response_format=ArticleInfo,
    agent_name="article_extractor"
)

# Display results
print("="*70)
print("EJERCICIO 1: EXTRACCIÓN DE INFORMACIÓN CLAVE")
print("="*70)
print("\nResultado estructurado:\n")
print(f"Título: {article_info.title}")
print(f"Tema: {article_info.main_topic}")
print(f"Sentimiento: {article_info.sentiment}")
print(f"\nResumen:\n{article_info.summary}")
print(f"\nPuntos clave:")
for i, point in enumerate(article_info.key_points, 1):
    print(f"  {i}. {point}")
print(f"\nEntidades mencionadas: {', '.join(article_info.entities)}")
if article_info.date_mentioned:
    print(f"Fecha: {article_info.date_mentioned}")

print("\n" + "-"*70)
print("\nJSON completo:")
print(json.dumps(article_info.model_dump(), indent=2, ensure_ascii=False))


ServiceResponseException: <class 'agent_framework.openai._chat_client.OpenAIChatClient'> service failed to complete the prompt: Unauthorized

---

## Ejercicio 2: Generación de objetos Python desde texto

En este ejercicio, crearemos objetos Python completos a partir de descripciones en lenguaje natural. El agente generará instancias de clases con tipos de datos nativos de Python.

In [ ]:
# Define classes to represent employees
class Address(BaseModel):
    """Dirección física"""
    street: str = Field(description="Calle y número")
    city: str = Field(description="Ciudad")
    postal_code: str = Field(description="Código postal")
    country: str = Field(description="País")


class Department(str, Enum):
    """Departamentos de la empresa"""
    ENGINEERING = "engineering"
    SALES = "sales"
    MARKETING = "marketing"
    HR = "human_resources"
    FINANCE = "finance"
    OPERATIONS = "operations"


class Employee(BaseModel):
    """Información de un empleado"""
    employee_id: int = Field(description="ID único del empleado")
    first_name: str = Field(description="Nombre")
    last_name: str = Field(description="Apellido")
    email: str = Field(description="Email corporativo")
    phone: str = Field(description="Teléfono de contacto")
    department: Department = Field(description="Departamento")
    position: str = Field(description="Cargo o posición")
    salary: float = Field(description="Salario anual en euros")
    hire_date: str = Field(description="Fecha de contratación (YYYY-MM-DD)")
    address: Address = Field(description="Dirección del empleado")
    is_remote: bool = Field(description="¿Trabaja de forma remota?")
    skills: List[str] = Field(description="Habilidades técnicas o profesionales")


# Employee description in free text
employee_description = """
María Carmen García López fue contratada el 15 de marzo de 2023 como Ingeniera Senior 
de Software. Su ID de empleado es 10542. Trabaja en el departamento de ingeniería con 
un salario anual de 65000 euros.

Puede contactarla en mc.garcia@empresa.com o al teléfono +34 666 123 456. 
Actualmente trabaja desde Madrid de forma remota. Su dirección registrada es 
Calle Gran Vía 28, Madrid, código postal 28013, España.

María tiene experiencia en Python, JavaScript, React, Django y bases de datos SQL. 
También domina metodologías ágiles y arquitectura de microservicios.
"""

# Instructions for the agent
instructions_employee = """Eres un experto en procesamiento de información de recursos humanos.

Analiza la descripción del empleado y extrae TODA la información disponible.

Reglas importantes:
- El employee_id debe ser numérico
- El email debe tener formato válido
- El departamento debe ser uno de los valores válidos del enum
- La fecha debe estar en formato YYYY-MM-DD
- El salario debe ser numérico (sin símbolos de moneda)
- Identifica correctamente si trabaja remoto o presencial

Responde SOLO con el JSON estructurado."""

# Run the agent
employee = await run_structured_agent(
    instructions=instructions_employee,
    user_input=employee_description,
    response_format=Employee,
    agent_name="employee_parser"
)

# Display results
print("="*70)
print("EJERCICIO 2: GENERACIÓN DE OBJETOS PYTHON")
print("="*70)
print("\nObjeto Employee creado:\n")
print(f"ID: {employee.employee_id}")
print(f"Nombre completo: {employee.first_name} {employee.last_name}")
print(f"Email: {employee.email}")
print(f"Teléfono: {employee.phone}")
print(f"Departamento: {employee.department.value}")
print(f"Posición: {employee.position}")
print(f"Salario: {employee.salary:,.2f} EUR")
print(f"Fecha de contratación: {employee.hire_date}")
print(f"Trabajo remoto: {'Sí' if employee.is_remote else 'No'}")
print(f"\nDirección:")
print(f"  {employee.address.street}")
print(f"  {employee.address.postal_code} {employee.address.city}")
print(f"  {employee.address.country}")
print(f"\nHabilidades: {', '.join(employee.skills)}")

print("\n" + "-"*70)
print("\nTipo de objeto:", type(employee))
print("Es instancia de Employee:", isinstance(employee, Employee))
print("\nJSON serializado:")
print(json.dumps(employee.model_dump(), indent=2, ensure_ascii=False))

# Demonstrate that we can use the object in Python code
print("\n" + "-"*70)
print("\nEjemplo de uso en código Python:")
print(f"Salario mensual: {employee.salary / 12:,.2f} EUR")
print(f"Email dominio: {employee.email.split('@')[1]}")
print(f"Años de experiencia requerida: {len(employee.skills)} habilidades registradas")
